# Jリーグ観客動員数予測コンテスト　特徴量エンジニアリング

In [229]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from pandas.tseries.offsets import *
%matplotlib inline

### データ読込

In [230]:
train = pd.read_csv("train.csv" )
test = pd.read_csv("test.csv") 
holidays_in_japan = pd.read_csv("holidays_in_japan.csv" )
match_reports= pd.read_csv("match_reports.csv" )
stadium_capacity_mapping= pd.read_csv("stadium_capacity_mapping.csv" )

#### 外部データ読込

In [231]:
ex_stadium_capacity_mapping= pd.read_csv("ex_stadium_capacity_mapping.csv" , encoding="shift_jis")
ex_compose_reports= pd.read_csv("ex_total.csv" , encoding="shift_jis")

In [232]:
ex_match_reports= pd.read_csv("ex_match_reports_mini.csv")

In [233]:
ex_compose_reports.shape

(14424, 14)

In [234]:
ex_compose_reports.columns

Index(['attendance', 'away_team', 'broadcasters', 'division', 'home_team',
       'id', 'kick_off_time', 'match_date', 'round', 'section', 'humidity',
       'temperature', 'venue', 'weather'],
      dtype='object')

In [235]:
train.columns

Index(['id', 'match_date', 'kick_off_time', 'section', 'round', 'home_team',
       'away_team', 'venue', 'weather', 'temperature', 'humidity',
       'broadcasters', 'attendance'],
      dtype='object')

divisionが不明　データ点数はほぼ同じ　二つに分けているだけか？  
→11/10　おそらく節が2分割かどうかのデータ

In [236]:
ex_compose_reports["division"].value_counts()

1    7350
2    7074
Name: division, dtype: int64

#### 外部データのdivisionは不明・かつ,train/testにないので削除 

In [237]:
ex_compose_reports = ex_compose_reports.drop("division" , axis=1)

### 外部データから抽出・昔のデータ取得する

In [238]:
ex_compose_reports =  ex_compose_reports.loc[(pd.to_datetime(ex_compose_reports["match_date"]) < pd.to_datetime("2006-3-31")),:]

## データをまとめていじれるようにtrain,testデータを統合

In [239]:
compose = pd.concat((ex_compose_reports,train,test),axis=0)
#compose = pd.concat((train,test),axis=0)
compose = compose.reset_index().copy() #indexコピー
print(train.shape)
print(test.shape)
print(ex_compose_reports.shape)
print(compose.shape)

(3366, 13)
(477, 12)
(5167, 13)
(9010, 14)


In [240]:
#重複行削除
ex_compose_reports2= ex_compose_reports.drop_duplicates(inplace = False,subset="id")

#重複行削除
train2= train.drop_duplicates(inplace = False,subset="id")

#重複行削除
test2= test.drop_duplicates(inplace = False,subset="id")

In [241]:
print(train2.shape)
print(test2.shape)
print(ex_compose_reports2.shape)
print(compose.shape)

(3366, 13)
(477, 12)
(5167, 13)
(9010, 14)


In [242]:
train.columns

Index(['id', 'match_date', 'kick_off_time', 'section', 'round', 'home_team',
       'away_team', 'venue', 'weather', 'temperature', 'humidity',
       'broadcasters', 'attendance'],
      dtype='object')

In [243]:
print(ex_compose_reports2.columns)

Index(['attendance', 'away_team', 'broadcasters', 'home_team', 'id',
       'kick_off_time', 'match_date', 'round', 'section', 'humidity',
       'temperature', 'venue', 'weather'],
      dtype='object')


### 欠損データの確認

In [244]:
# 欠損データの確認 missing_values_tableを作る
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum()/len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns= {0 : 'Missing Values', 1: '% of total values'})
    return mis_val_table_ren_columns

In [245]:
missing_values_table(compose)

,Missing Values,% of total values
index,0,0.000000
attendance,477,5.294118
away_team,0,0.000000
broadcasters,1765,19.589345
home_team,0,0.000000
humidity,198,2.197558
id,0,0.000000
kick_off_time,0,0.000000
match_date,0,0.000000
round,0,0.000000


2018年　第33節　第34節の天気系がデータがない
→そもそも、この部分は予測する部分なのでないよね。
年によって違うが傾向は似ているので、それで対応する？

broadcastのnanデータは、そもそも放送していない

## スタジアムデータ統合 外部データと統合する

In [246]:
ex_stadium_capacity_mapping.columns = ["stadium","capacity"]
compose_stadium_capacity_mapping = pd.concat((ex_stadium_capacity_mapping,stadium_capacity_mapping),axis=0)
compose_stadium_capacity_mapping = compose_stadium_capacity_mapping.reset_index().copy() #indexコピー
print(compose_stadium_capacity_mapping.shape)

(265, 3)


In [247]:
print(compose_stadium_capacity_mapping["stadium"].value_counts())
print(len(compose_stadium_capacity_mapping["stadium"].value_counts()))
#本来の要素は190ある

ノエビアスタジアム神戸             2
岐阜メモリアルセンター長良川競技場       2
パナソニックスタジアム吹田           2
パロマ瑞穂スタジアム              2
熊谷スポーツ文化公園陸上競技場         2
札幌ドーム                   2
アウトソーシングスタジアム日本平        2
東北電力ビッグスワンスタジアム         2
山梨県小瀬スポーツ公園陸上競技場        2
札幌厚別公園競技場               2
横浜市三ツ沢公園陸上競技場           2
味の素フィールド西が丘             2
レベルファイブスタジアム            2
横浜市三ツ沢公園球技場             2
フクダ電子アリーナ               2
名古屋市瑞穂陸上競技場             2
長崎市総合運動公園かきどまり陸上競技場     2
石川県西部緑地公園陸上競技場          2
栃木県グリーンスタジアム            2
トランスコスモススタジアム長崎         2
ニンジニアスタジアム              2
ベストアメニティスタジアム           2
名古屋市瑞穂球技場               2
とりぎんバードスタジアム            2
味の素スタジアム                2
新潟スタジアム                 2
熊本市水前寺競技場               2
佐賀県総合運動場陸上競技場           2
さいたま市浦和駒場スタジアム          2
三協フロンテア柏スタジアム           2
                       ..
福山市竹ケ端運動公園陸上競技場         1
ヤマハスタジアム（磐田）            1
岐阜メモリアルセンター長良川球技メドウ     1
ミクニワールドスタジアム北九州         1
国立スポーツ科学センター西が丘サッカー場    1
神戸ウイングスタジアム             1
佐伯市営陸上競技場               1
ｋａｎｋｏスタジアム  

スタジアムのデータを統合したが、重複しているデータが存在してる  
重複データを削除して,indexもリセットする

In [248]:
#邪魔な列を削除する
compose_stadium_capacity_mapping = compose_stadium_capacity_mapping.drop("index" , axis=1)

#重複行削除
compose_stadium_capacity_mapping.drop_duplicates(inplace = True,subset="stadium")

#indexリセット
compose_stadium_capacity_mapping = compose_stadium_capacity_mapping.reset_index()

#index削除
compose_stadium_capacity_mapping = compose_stadium_capacity_mapping.drop("index" , axis=1)

###  capacityのnan補間

In [249]:
cap = {"横浜市三ツ沢総合公園球技場":15454, "広島スタジアム":33000, "名古屋市瑞穂球技場":15000, "広島ビッグアーチ":35000, 
       "静岡県営草薙陸上競技場":28000, "等々力緑地運動公園陸上競技場":27495, "三重県営鈴鹿スポーツガーデン":12000, 
       "テクノポート福井スタジアム":21053, "埼玉県営大宮公園サッカー場":15500, "秋田市営八橋陸上競技場":20125, 
       "七北田公園仙台スタジアム":19694, "札幌ドーム":41484, "神戸ウイングスタジアム":34000, "新潟スタジアム":42300, 
       "国立競技場":54224, "エディオンスタジアム広島":35000, "長野運動公園総合運動場":17200, "ひたちなか市総合運動公園陸上競技場":15000, 
       "都市公園富士北麓公園陸上競技場":11105, "長野市営長野運動公園総合運動場":17200, "大津市皇子山総合運動公園陸上競技場":12000,
       "群馬県立敷島公園県営サッカー・ラグビー場":7800,
       "Ｊヴィレッジスタジアム":5000,"岐阜メモリアルセンター長良川球技メドウ":3560}

In [250]:
#dataframeに変換
cap_se = pd.Series(cap)
cap_df = pd.DataFrame(cap_se)
cap_df.columns = ["capacity"]

In [251]:
cap_df["stadium"] = cap_df.index.values
cap_df.head()

#indexリセット
cap_df = cap_df.reset_index()
#index 削除
cap_df = cap_df.drop("index" ,axis=1)

In [252]:
#merge stadiumをkeyにする capacityで名前が被っているので、後でひとつにまとめる
compose_stadium_capacity_mapping2 = pd.merge(compose_stadium_capacity_mapping, cap_df , on="stadium" ,how="left")
#nanを0に変換
compose_stadium_capacity_mapping2 =compose_stadium_capacity_mapping2.fillna(0)
#capacityをひとつにまとめる
compose_stadium_capacity_mapping2["capacity"] =compose_stadium_capacity_mapping2["capacity_x"].astype(int) +compose_stadium_capacity_mapping2["capacity_y"].astype(int) 
#不要な列を削除する
compose_stadium_capacity_mapping2 = compose_stadium_capacity_mapping2.drop("capacity_x",axis=1)
compose_stadium_capacity_mapping2 = compose_stadium_capacity_mapping2.drop("capacity_y",axis=1)

In [253]:
compose_stadium_capacity_mapping2.to_csv("compose_stadium_capacity_mapping.csv",header=True,index=None)

## 試合結果統合

In [254]:
#それぞれの配列に列追加
match_reports["home_team_win"] = match_reports["home_team_score"] > match_reports["away_team_score"]
ex_match_reports["home_team_win"] = ex_match_reports["home_team_score"] > ex_match_reports["away_team_score"]

In [255]:
#必要な列だけ抽出
match_reports_mini = match_reports.loc[:,['id',"home_team_win", "home_team_score", "away_team_score"]]
compose_match_reports = pd.concat((ex_match_reports,match_reports_mini),axis=0)
compose_match_reports = compose_match_reports.reset_index().copy() #indexコピー
compose_match_reports = compose_match_reports.drop("index",axis=1)
print(compose_match_reports.shape)

(18201, 4)


In [256]:
#重複行削除
compose_match_reports2 = compose_match_reports.drop_duplicates(inplace = False,subset="id")

In [257]:
compose_match_reports2.shape

(14376, 4)

In [258]:
compose_match_reports2["id"] = compose_match_reports2["id"].astype(str)

C:\Users\Mustang8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



## 特徴量作成・数値変換

In [259]:
compose_processing = compose.copy() #加工データ用にcopy
print(compose_processing.shape)

(9010, 14)


In [260]:
compose_processing

,index,attendance,away_team,broadcasters,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather
0,0,59626.0,横浜M,NaN,Ｖ川崎,NaN,49,19:29,1993-05-15,第１日,第１節,NaN,国立霞ヶ丘競技場,晴
1,1,14126.0,清水,NaN,横浜Ｆ,NaN,50,13:05,1993-05-16,第２日,第１節,NaN,横浜市三ツ沢総合公園球技場,晴
2,2,11875.0,市原,NaN,広島,NaN,51,13:59,1993-05-16,第２日,第１節,NaN,広島スタジアム,晴
3,3,10898.0,名古屋,NaN,鹿島,NaN,52,16:00,1993-05-16,第２日,第１節,NaN,県立カシマサッカースタジアム,晴
4,4,19580.0,浦和,NaN,Ｇ大阪,NaN,53,19:04,1993-05-16,第２日,第１節,NaN,万博記念競技場,晴
5,5,9183.0,広島,NaN,清水,NaN,58,18:58,1993-05-19,第１日,第２節,NaN,日本平運動公園球技場,曇
6,6,9259.0,名古屋,NaN,浦和,NaN,56,19:00,1993-05-19,第１日,第２節,NaN,浦和市駒場スタジアム,晴
7,7,10332.0,Ｇ大阪,NaN,横浜M,NaN,57,19:00,1993-05-19,第１日,第２節,NaN,横浜市三ツ沢総合公園球技場,晴
8,8,9403.0,横浜Ｆ,NaN,鹿島,NaN,54,19:01,1993-05-19,第１日,第２節,NaN,県立カシマサッカースタジアム,晴
9,9,46959.0,Ｖ川崎,NaN,市原,NaN,55,19:02,1993-05-19,第１日,第２節,NaN,国立霞ヶ丘競技場,晴


### 時間の変数を利用できるようにする

In [261]:
import datetime
compose_processing["match_date"] = pd.to_datetime(compose_processing["match_date"])
compose_processing["kick_off_time"] = pd.to_datetime(compose_processing["kick_off_time"])

In [262]:
####################################
## 列の追加
compose_processing["match_date_year"] = 0
compose_processing["match_date_month"] = 0
compose_processing["match_date_day"] = 0
################################
compose_processing["match_date_year"]  = compose_processing["match_date"].dt.year
compose_processing["match_date_month"] = compose_processing["match_date"].dt.month
compose_processing["match_date_day"]   = compose_processing["match_date"].dt.day


#キックオフの時間の追加
compose_processing["kick_off_time_hour"] =0
compose_processing["kick_off_time_hour"] = compose_processing["kick_off_time"].dt.hour
#######################################################################################
compose_processing["kick_off_time_minute"] =0
compose_processing["kick_off_time_minute"] =compose_processing["kick_off_time"].dt.minute

#曜日の追加
compose_processing["match_date_weekday"] = 0
compose_processing["match_date_weekday"] = compose_processing["match_date"].dt.dayofweek
############################################################################

#kick-off_timeのデータは使わないので削除する
compose_processing = compose_processing.drop("kick_off_time", axis =1)

#### 祝日追加

In [263]:
import jpholiday

In [215]:
holidays = jpholiday.holidays(datetime.date(1993, 1, 1), datetime.date(2018, 12, 31))

In [216]:
holidays

[(datetime.date(1993, 1, 1), '元日'),
 (datetime.date(1993, 1, 15), '成人の日'),
 (datetime.date(1993, 2, 11), '建国記念の日'),
 (datetime.date(1993, 3, 20), '春分の日'),
 (datetime.date(1993, 4, 29), 'みどりの日'),
 (datetime.date(1993, 5, 3), '憲法記念日'),
 (datetime.date(1993, 5, 4), '国民の休日'),
 (datetime.date(1993, 5, 5), 'こどもの日'),
 (datetime.date(1993, 6, 9), '皇太子・皇太子徳仁親王の結婚の儀'),
 (datetime.date(1993, 9, 15), '敬老の日'),
 (datetime.date(1993, 9, 23), '秋分の日'),
 (datetime.date(1993, 10, 10), '体育の日'),
 (datetime.date(1993, 10, 11), '体育の日 振替休日'),
 (datetime.date(1993, 11, 3), '文化の日'),
 (datetime.date(1993, 11, 23), '勤労感謝の日'),
 (datetime.date(1993, 12, 23), '天皇誕生日'),
 (datetime.date(1994, 1, 1), '元日'),
 (datetime.date(1994, 1, 15), '成人の日'),
 (datetime.date(1994, 2, 11), '建国記念の日'),
 (datetime.date(1994, 3, 21), '春分の日'),
 (datetime.date(1994, 4, 29), 'みどりの日'),
 (datetime.date(1994, 5, 3), '憲法記念日'),
 (datetime.date(1994, 5, 4), '国民の休日'),
 (datetime.date(1994, 5, 5), 'こどもの日'),
 (datetime.date(1994, 9, 15), '敬老の日'),
 (

In [217]:
holidays_se = pd.Series(holidays)
holidays_df = pd.DataFrame(holidays_se)

In [220]:
holidays_df.columns = ["holiday"]

In [222]:
#compose_processingにholidayを追加するため
#mergeするために,列名を同じ名前にする .rename
holidays_df  = holidays_df .rename(columns={"holiday":"match_date"})

In [225]:
holidays_df= list(holidays_df)

In [228]:
compose_processing=pd.merge(compose_processing,holidays_df ,how = "left" , on = "match_date")

ValueError: can not merge DataFrame with instance of type <class 'list'>

In [ ]:



#ユニーク数カウント
description_number =compose_processing["description"].drop_duplicates().nunique()

#休日の辞書作成
holiday_dict={}
for description_num in range(description_number+1):
    holiday_dict.update({compose_processing["description"].drop_duplicates().unique()[description_num]:description_num})
#置換
compose_processing["description"] =compose_processing["description"].replace(holiday_dict)

### 1.チーム名　ラベリング  
チーム数がhome_team とaway_teamで同じ数・かつ、同じラベルにしたいので統一してラベリングを行う  
データを見たところ、同じチームだが、全角・半角があるので、それを統合してからラベリングを行う

In [264]:
#チーム一覧 43チームある
print(compose_processing["home_team"].drop_duplicates())
print(compose_processing["home_team"].drop_duplicates().unique())

0        Ｖ川崎
1        横浜Ｆ
2         広島
3         鹿島
4        Ｇ大阪
5         清水
6         浦和
7        横浜M
9         市原
10       名古屋
188       磐田
189       平塚
455        柏
457      Ｃ大阪
816       福岡
822       京都
1063      神戸
1329      札幌
1632    横浜FM
1876     Ｆ東京
1881     川崎Ｆ
2116     東京Ｖ
2348      仙台
2596      大分
2838      新潟
3076      大宮
3081      千葉
3373      甲府
3423      鳥栖
3431      山形
3601      湘南
3603      水戸
3823    横浜FC
4873      草津
4881      徳島
5140      愛媛
5167     G大阪
5169    FC東京
5173     川崎F
5184     C大阪
5796     東京V
7938      松本
8855      長崎
Name: home_team, dtype: object
['Ｖ川崎' '横浜Ｆ' '広島' '鹿島' 'Ｇ大阪' '清水' '浦和' '横浜M' '市原' '名古屋' '磐田' '平塚' '柏'
 'Ｃ大阪' '福岡' '京都' '神戸' '札幌' '横浜FM' 'Ｆ東京' '川崎Ｆ' '東京Ｖ' '仙台' '大分' '新潟' '大宮'
 '千葉' '甲府' '鳥栖' '山形' '湘南' '水戸' '横浜FC' '草津' '徳島' '愛媛' 'G大阪' 'FC東京' '川崎F'
 'C大阪' '東京V' '松本' '長崎']


http://heatrock.fc2web.com/j_hensen.html  
かつ、チーム名が歴史と共に変遷している  
V川崎→東京V  
市原→千葉  
横浜F→横浜FM  
横浜M→横浜FM(合併した)  
平塚→湘南  
F東京というチームは存在しない→おそらくFC東京

チームのラベリングは統一すべきだと考えるので統一する

In [265]:
compose_processing["home_team"] = compose_processing["home_team"].replace('東京Ｖ',"東京V")
compose_processing["away_team"]  = compose_processing["away_team"].replace('東京Ｖ',"東京V")

compose_processing["home_team"] = compose_processing["home_team"].replace("Ｖ川崎","東京V")
compose_processing["away_team"] = compose_processing["away_team"].replace("Ｖ川崎","東京V")

compose_processing["home_team"] = compose_processing["home_team"].replace('市原','千葉')
compose_processing["away_team"] = compose_processing["away_team"].replace('市原','千葉')

compose_processing["home_team"] = compose_processing["home_team"].replace('平塚', '湘南')
compose_processing["away_team"] = compose_processing["away_team"].replace('平塚', '湘南')

compose_processing["home_team"] = compose_processing["home_team"].replace('横浜M', '横浜FM' )
compose_processing["away_team"] = compose_processing["away_team"].replace('横浜M', '横浜FM' )

compose_processing["home_team"] = compose_processing["home_team"].replace('横浜Ｆ', '横浜FM' )
compose_processing["away_team"] = compose_processing["away_team"].replace('横浜Ｆ', '横浜FM' )

compose_processing["home_team"] = compose_processing["home_team"].replace('Ｇ大阪', 'G大阪' )
compose_processing["away_team"] = compose_processing["away_team"].replace('Ｇ大阪', 'G大阪' )

compose_processing["home_team"] = compose_processing["home_team"].replace('Ｃ大阪', 'C大阪' )
compose_processing["away_team"] = compose_processing["away_team"].replace('Ｃ大阪', 'C大阪' )

compose_processing["home_team"] = compose_processing["home_team"].replace('Ｆ東京' ,  'FC東京' )
compose_processing["away_team"] = compose_processing["away_team"].replace('Ｆ東京' ,  'FC東京')

compose_processing["home_team"] = compose_processing["home_team"].replace('川崎Ｆ'  ,  '川崎F'  )
compose_processing["away_team"] = compose_processing["away_team"].replace('川崎Ｆ'  ,  '川崎F' )

In [266]:
#チーム一覧 33チームある
print(compose_processing["home_team"].drop_duplicates())
print(compose_processing["home_team"].drop_duplicates().unique())

0        東京V
1       横浜FM
2         広島
3         鹿島
4        G大阪
5         清水
6         浦和
9         千葉
10       名古屋
188       磐田
189       湘南
455        柏
457      C大阪
816       福岡
822       京都
1063      神戸
1329      札幌
1876    FC東京
1881     川崎F
2348      仙台
2596      大分
2838      新潟
3076      大宮
3373      甲府
3423      鳥栖
3431      山形
3603      水戸
3823    横浜FC
4873      草津
4881      徳島
5140      愛媛
7938      松本
8855      長崎
Name: home_team, dtype: object
['東京V' '横浜FM' '広島' '鹿島' 'G大阪' '清水' '浦和' '千葉' '名古屋' '磐田' '湘南' '柏' 'C大阪'
 '福岡' '京都' '神戸' '札幌' 'FC東京' '川崎F' '仙台' '大分' '新潟' '大宮' '甲府' '鳥栖' '山形' '水戸'
 '横浜FC' '草津' '徳島' '愛媛' '松本' '長崎']


In [267]:
print(compose_processing["home_team"].drop_duplicates().unique())

['東京V' '横浜FM' '広島' '鹿島' 'G大阪' '清水' '浦和' '千葉' '名古屋' '磐田' '湘南' '柏' 'C大阪'
 '福岡' '京都' '神戸' '札幌' 'FC東京' '川崎F' '仙台' '大分' '新潟' '大宮' '甲府' '鳥栖' '山形' '水戸'
 '横浜FC' '草津' '徳島' '愛媛' '松本' '長崎']


In [268]:
#数をカウントする
team_number = compose_processing["home_team"].drop_duplicates().nunique()
#チーム名の辞書作成
team_dict={}
for team_num in range(team_number):
    team_dict.update({compose_processing["home_team"].drop_duplicates().unique()[team_num]:team_num})

#置換
compose_processing["home_team"] =compose_processing["home_team"].replace(team_dict)
compose_processing["away_team"] =compose_processing["away_team"].replace(team_dict)

In [269]:
team_dict

{'C大阪': 12,
 'FC東京': 17,
 'G大阪': 4,
 '京都': 14,
 '仙台': 19,
 '千葉': 7,
 '名古屋': 8,
 '大分': 20,
 '大宮': 22,
 '山形': 25,
 '川崎F': 18,
 '広島': 2,
 '徳島': 29,
 '愛媛': 30,
 '新潟': 21,
 '札幌': 16,
 '東京V': 0,
 '松本': 31,
 '柏': 11,
 '横浜FC': 27,
 '横浜FM': 1,
 '水戸': 26,
 '浦和': 6,
 '清水': 5,
 '湘南': 10,
 '甲府': 23,
 '磐田': 9,
 '神戸': 15,
 '福岡': 13,
 '草津': 28,
 '長崎': 32,
 '鳥栖': 24,
 '鹿島': 3}

In [270]:
compose_processing.to_csv("compose_processing_team_conversion.csv",header=True,index=None)

In [271]:
compose_processing.head()

,index,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,venue,weather,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday
0,0,59626.0,1,NaN,0,NaN,49,1993-05-15,第１日,第１節,NaN,国立霞ヶ丘競技場,晴,1993,5,15,19,29,5
1,1,14126.0,5,NaN,1,NaN,50,1993-05-16,第２日,第１節,NaN,横浜市三ツ沢総合公園球技場,晴,1993,5,16,13,5,6
2,2,11875.0,7,NaN,2,NaN,51,1993-05-16,第２日,第１節,NaN,広島スタジアム,晴,1993,5,16,13,59,6
3,3,10898.0,8,NaN,3,NaN,52,1993-05-16,第２日,第１節,NaN,県立カシマサッカースタジアム,晴,1993,5,16,16,0,6
4,4,19580.0,6,NaN,4,NaN,53,1993-05-16,第２日,第１節,NaN,万博記念競技場,晴,1993,5,16,19,4,6


### 2.ラウンド

In [272]:
compose_processing["round"].drop_duplicates().unique()

array(['第１日', '第２日', '第３日', '第４日', '第1日', '第2日', '第3日', '第4日'],
      dtype=object)

半角・全角入り混じっている

In [273]:
compose_processing["round"] = compose_processing["round"].replace('第１日','第1日')
compose_processing["round"] = compose_processing["round"].replace( '第２日','第2日')
compose_processing["round"] = compose_processing["round"].replace(  '第３日','第3日')
compose_processing["round"] = compose_processing["round"].replace( '第４日','第4日')

In [274]:
compose_processing["round"].drop_duplicates().unique()

array(['第1日', '第2日', '第3日', '第4日'], dtype=object)

In [275]:
round_number = compose_processing["round"].drop_duplicates().nunique()
#ラウンド辞書作成
round_dict={}
for round_num in range(round_number):
    round_dict.update({compose_processing["round"].drop_duplicates().unique()[round_num]:round_num})
#置換
compose_processing["round"] = compose_processing["round"].replace(round_dict)

In [276]:
round_dict

{'第1日': 0, '第2日': 1, '第3日': 2, '第4日': 3}

### 3.セクション

In [277]:
compose_processing["section"].drop_duplicates().unique()

array(['第１節', '第２節', '第３節', '第４節', '第５節', '第６節', '第７節', '第８節', '第９節',
       '第１０節', '第１１節', '第１２節', '第１３節', '第１４節', '第１５節', '第１６節', '第１７節',
       '第１８節', '第１９節', '第２０節', '第２１節', '第２２節', '第２３節', '第２４節', '第２５節',
       '第２６節', '第２７節', '第２８節', '第２９節', '第３０節', '第３１節', '第３２節', '第３３節',
       '第３４節', '第３５節', '第３６節', '第３７節', '第３８節', '第３９節', '第４０節', '第４１節',
       '第４２節', '第４３節', '第４４節', '第1節', '第2節', '第3節', '第4節', '第5節', '第6節',
       '第7節', '第8節', '第9節', '第10節', '第11節', '第12節', '第13節', '第14節',
       '第15節', '第16節', '第17節', '第18節', '第19節', '第20節', '第21節', '第22節',
       '第23節', '第24節', '第25節', '第26節', '第27節', '第28節', '第29節', '第30節',
       '第31節', '第32節', '第33節', '第34節'], dtype=object)

半角・全角入り混じっている

In [278]:
compose_processing["section"] = compose_processing["section"].replace('第１節','第1節')
compose_processing["section"] = compose_processing["section"].replace('第２節','第2節')
compose_processing["section"] = compose_processing["section"].replace('第３節','第3節')
compose_processing["section"] = compose_processing["section"].replace('第４節','第4節')
compose_processing["section"] = compose_processing["section"].replace('第５節','第5節')
compose_processing["section"] = compose_processing["section"].replace('第６節','第6節')
compose_processing["section"] = compose_processing["section"].replace('第７節','第7節')
compose_processing["section"] = compose_processing["section"].replace('第８節','第8節')
compose_processing["section"] = compose_processing["section"].replace('第９節','第9節')
compose_processing["section"] = compose_processing["section"].replace('第１０節','第10節')

compose_processing["section"] = compose_processing["section"].replace('第１１節','第11節')
compose_processing["section"] = compose_processing["section"].replace('第１２節','第12節')
compose_processing["section"] = compose_processing["section"].replace('第１３節','第13節')
compose_processing["section"] = compose_processing["section"].replace('第１４節','第14節')
compose_processing["section"] = compose_processing["section"].replace('第１５節','第15節')
compose_processing["section"] = compose_processing["section"].replace('第１６節','第16節')
compose_processing["section"] = compose_processing["section"].replace('第１７節','第17節')
compose_processing["section"] = compose_processing["section"].replace('第１８節','第18節')
compose_processing["section"] = compose_processing["section"].replace('第１９節','第19節')
compose_processing["section"] = compose_processing["section"].replace('第２０節','第20節')

compose_processing["section"] = compose_processing["section"].replace('第２１節','第21節')
compose_processing["section"] = compose_processing["section"].replace('第２２節','第22節')
compose_processing["section"] = compose_processing["section"].replace('第２３節','第23節')
compose_processing["section"] = compose_processing["section"].replace('第２４節','第24節')
compose_processing["section"] = compose_processing["section"].replace('第２５節','第25節')
compose_processing["section"] = compose_processing["section"].replace('第２６節','第26節')
compose_processing["section"] = compose_processing["section"].replace('第２７節','第27節')
compose_processing["section"] = compose_processing["section"].replace('第２８節','第28節')
compose_processing["section"] = compose_processing["section"].replace('第２９節','第29節')
compose_processing["section"] = compose_processing["section"].replace('第３０節','第30節')

compose_processing["section"] = compose_processing["section"].replace('第３１節','第31節')
compose_processing["section"] = compose_processing["section"].replace('第３２節','第32節')
compose_processing["section"] = compose_processing["section"].replace('第３３節','第33節')
compose_processing["section"] = compose_processing["section"].replace('第３４節','第34節')
compose_processing["section"] = compose_processing["section"].replace('第３５節','第35節')
compose_processing["section"] = compose_processing["section"].replace('第３６節','第36節')
compose_processing["section"] = compose_processing["section"].replace('第３７節','第37節')
compose_processing["section"] = compose_processing["section"].replace('第３８節','第38節')
compose_processing["section"] = compose_processing["section"].replace('第３９節','第39節')
compose_processing["section"] = compose_processing["section"].replace('第４０節','第40節')

compose_processing["section"] = compose_processing["section"].replace('第４１節','第41節')
compose_processing["section"] = compose_processing["section"].replace('第４２節','第42節')
compose_processing["section"] = compose_processing["section"].replace('第４３節','第43節')
compose_processing["section"] = compose_processing["section"].replace('第４４節','第44節')

In [279]:
compose_processing["section"].drop_duplicates().unique()

array(['第1節', '第2節', '第3節', '第4節', '第5節', '第6節', '第7節', '第8節', '第9節',
       '第10節', '第11節', '第12節', '第13節', '第14節', '第15節', '第16節', '第17節',
       '第18節', '第19節', '第20節', '第21節', '第22節', '第23節', '第24節', '第25節',
       '第26節', '第27節', '第28節', '第29節', '第30節', '第31節', '第32節', '第33節',
       '第34節', '第35節', '第36節', '第37節', '第38節', '第39節', '第40節', '第41節',
       '第42節', '第43節', '第44節'], dtype=object)

セクション
1993 1-18 1-18  　　
1994 1-22 1-22  
1995 1-26 1-26  
1997 1-17 10月に試合終了
1998　1-17　1-17  
1999　1-15　1-15  
2000　1-15　1-15  
2001　1-15　1-15  
2002　1-15　1-15  
2003　1-15　1-15  
2004　1-15　1-15 
なので、足す必要がある

### 工事中　節をそろえる必要がある

In [186]:
if((compose_processing["match_date"])>= pd.to_datetime("1993-5-15")) &( (compose_processing["match_date"]) < pd.to_datetime("2017-2-24"))

0        True
1        True
2        True
3        True
4        True
5        True
6        True
7        True
8        True
9        True
10       True
11       True
12       True
13       True
14       True
15       True
16       True
17       True
18       True
19       True
20       True
21       True
22       True
23       True
24       True
25       True
26       True
27       True
28       True
29       True
        ...  
8980    False
8981    False
8982    False
8983    False
8984    False
8985    False
8986    False
8987    False
8988    False
8989    False
8990    False
8991    False
8992    False
8993    False
8994    False
8995    False
8996    False
8997    False
8998    False
8999    False
9000    False
9001    False
9002    False
9003    False
9004    False
9005    False
9006    False
9007    False
9008    False
9009    False
Name: match_date, Length: 9010, dtype: bool

In [179]:
compose_processing.loc[(compose_processing["match_date"]) < pd.to_datetime("2017-2-24"),:]

0       1993
1       1993
2       1993
3       1993
4       1993
5       1993
6       1993
7       1993
8       1993
9       1993
10      1993
11      1993
12      1993
13      1993
14      1993
15      1993
16      1993
17      1993
18      1993
19      1993
20      1993
21      1993
22      1993
23      1993
24      1993
25      1993
26      1993
27      1993
28      1993
29      1993
        ... 
8980    2018
8981    2018
8982    2018
8983    2018
8984    2018
8985    2018
8986    2018
8987    2018
8988    2018
8989    2018
8990    2018
8991    2018
8992    2018
8993    2018
8994    2018
8995    2018
8996    2018
8997    2018
8998    2018
8999    2018
9000    2018
9001    2018
9002    2018
9003    2018
9004    2018
9005    2018
9006    2018
9007    2018
9008    2018
9009    2018
Name: match_date_year, Length: 9010, dtype: int64

In [280]:
section_number = compose_processing["section"].drop_duplicates().nunique()

In [281]:
#第～節の辞書作成
section_dict ={}
for section_num in range(section_number):
    section_dict.update({compose_processing["section"].drop_duplicates().unique()[section_num]:section_num})
#置換
compose_processing["section"] =compose_processing["section"].replace(section_dict)

In [282]:
compose_processing

,index,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,venue,weather,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday
0,0,59626.0,1,NaN,0,NaN,49,1993-05-15,0,0,NaN,国立霞ヶ丘競技場,晴,1993,5,15,19,29,5
1,1,14126.0,5,NaN,1,NaN,50,1993-05-16,1,0,NaN,横浜市三ツ沢総合公園球技場,晴,1993,5,16,13,5,6
2,2,11875.0,7,NaN,2,NaN,51,1993-05-16,1,0,NaN,広島スタジアム,晴,1993,5,16,13,59,6
3,3,10898.0,8,NaN,3,NaN,52,1993-05-16,1,0,NaN,県立カシマサッカースタジアム,晴,1993,5,16,16,0,6
4,4,19580.0,6,NaN,4,NaN,53,1993-05-16,1,0,NaN,万博記念競技場,晴,1993,5,16,19,4,6
5,5,9183.0,2,NaN,5,NaN,58,1993-05-19,0,1,NaN,日本平運動公園球技場,曇,1993,5,19,18,58,2
6,6,9259.0,8,NaN,6,NaN,56,1993-05-19,0,1,NaN,浦和市駒場スタジアム,晴,1993,5,19,19,0,2
7,7,10332.0,4,NaN,1,NaN,57,1993-05-19,0,1,NaN,横浜市三ツ沢総合公園球技場,晴,1993,5,19,19,0,2
8,8,9403.0,1,NaN,3,NaN,54,1993-05-19,0,1,NaN,県立カシマサッカースタジアム,晴,1993,5,19,19,1,2
9,9,46959.0,0,NaN,7,NaN,55,1993-05-19,0,1,NaN,国立霞ヶ丘競技場,晴,1993,5,19,19,2,2


### 4.スタジアム

In [283]:
compose_processing["venue"].drop_duplicates()

0                       国立霞ヶ丘競技場
1                  横浜市三ツ沢総合公園球技場
2                        広島スタジアム
3                 県立カシマサッカースタジアム
4                        万博記念競技場
5                     日本平運動公園球技場
6                     浦和市駒場スタジアム
10                     名古屋市瑞穂球技場
11                      広島ビッグアーチ
13                   静岡県営草薙陸上競技場
19                等々力緑地運動公園陸上競技場
21                  鹿児島県立鴨池陸上競技場
31                     博多の森陸上競技場
32             神戸総合運動公園ユニバー記念競技場
33                       市原臨海競技場
40                   札幌厚別運動公園競技場
43                愛媛県総合運動公園陸上競技場
58                     熊本市水前寺競技場
60             岐阜メモリアルセンター長良川競技場
69                 栃木県立グリーンスタジアム
71                三重県営鈴鹿スポーツガーデン
74               長崎県立総合運動公園陸上競技場
105                    札幌厚別公園競技場
125                    岩手県営陸上競技場
135                 びんご運動公園陸上競技場
145                      宮城陸上競技場
159                    大分市営陸上競技場
162            京都市西京極総合運動公園陸上競技場
188              ジュビロ磐田サッカースタジアム
189                        平塚競技場
          

In [284]:
venue_number = compose_processing["venue"].drop_duplicates().nunique()#外部データありのとき＋１する nanデータ対応

In [285]:
#tcompose_processingにcapacityを追加するため
#mergeするために,列名を同じ名前にする .rename
compose_stadium_capacity_mapping2 = compose_stadium_capacity_mapping2.rename(columns={"stadium":"venue"})

In [286]:
#mergeする
compose_processing = pd.merge(compose_processing,compose_stadium_capacity_mapping2, on =["venue"],how="left")

In [287]:
#スタジアムの辞書の作成
venue_dict={}
add_venue_dict ={}
for venue_num in range(venue_number):
    venue_dict.update({compose_processing["venue"].drop_duplicates().unique()[venue_num]:venue_num})
    add_venue_dict.update({compose_processing["venue"].drop_duplicates().unique()[venue_num]:venue_num})

################################################################################################################
#置換
#stadium_capacity_mapping["stadium"] = stadium_capacity_mapping["stadium"].replace(add_venue_dict)
compose_processing["venue"] =compose_processing["venue"].replace(venue_dict)
########################################################################################################

In [288]:
compose_processing

,index,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,venue,weather,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,capacity
0,0,59626.0,1,NaN,0,NaN,49,1993-05-15,0,0,NaN,0,晴,1993,5,15,19,29,5,48000
1,1,14126.0,5,NaN,1,NaN,50,1993-05-16,1,0,NaN,1,晴,1993,5,16,13,5,6,15454
2,2,11875.0,7,NaN,2,NaN,51,1993-05-16,1,0,NaN,2,晴,1993,5,16,13,59,6,33000
3,3,10898.0,8,NaN,3,NaN,52,1993-05-16,1,0,NaN,3,晴,1993,5,16,16,0,6,40728
4,4,19580.0,6,NaN,4,NaN,53,1993-05-16,1,0,NaN,4,晴,1993,5,16,19,4,6,21000
5,5,9183.0,2,NaN,5,NaN,58,1993-05-19,0,1,NaN,5,曇,1993,5,19,18,58,2,20339
6,6,9259.0,8,NaN,6,NaN,56,1993-05-19,0,1,NaN,6,晴,1993,5,19,19,0,2,21500
7,7,10332.0,4,NaN,1,NaN,57,1993-05-19,0,1,NaN,1,晴,1993,5,19,19,0,2,15454
8,8,9403.0,1,NaN,3,NaN,54,1993-05-19,0,1,NaN,3,晴,1993,5,19,19,1,2,40728
9,9,46959.0,0,NaN,7,NaN,55,1993-05-19,0,1,NaN,0,晴,1993,5,19,19,2,2,48000


### 5. 天気について   
 気象庁の用語集(http://www.jma.go.jp/jma/kishou/know/yougo_hp/toki.html)によると  
 のち: 予報期間内の前と後で現象が異なるとき、その変化を示すときに用いる。  
 時々: 現象が断続的に起こり、その現象の発現期間の合計時間が予報期間の1/2未満のとき。  
一時: 現象が連続的に起こり、その現象の発現期間が予報期間の1/4未満のとき  

In [289]:
compose_processing["weather"].drop_duplicates()

0              晴
5              曇
10             雨
21          曇時々雨
30          曇一時晴
33          曇のち雨
38          曇のち晴
46          晴のち曇
55          曇一時雨
62          雨のち曇
72          曇時々晴
97          晴のち雨
151         晴時々曇
207         晴時々雪
322         晴一時雨
595         雨時々曇
638           雷雨
893         雨一時曇
1060        雨のち晴
1251       曇のち雷雨
1341        晴一時曇
1825     曇一時雨のち曇
1973       曇一時雷雨
2048       晴のち雷雨
2122        雪時々雨
2128        雨時々雪
2167        晴時々雨
2220          屋内
2422     晴のち曇のち雨
2430        晴のち霧
          ...   
3797     曇のち晴一時雨
3837        雨時々晴
3885    晴のち曇のち雷雨
4024     雨のち曇のち雨
4327        曇のち雪
4473     晴のち曇一時雨
4617        雪のち雨
4739       雷雨のち雨
4808          曇晴
4881        曇一時雪
4959     曇のち雨のち晴
5366    曇時々晴のち屋内
5431     雨のち曇時々雨
5489        曇時々雪
5660     曇のち雨一時雷
5916     晴時々曇一時雨
5947       雷雨のち曇
5949    曇一時雷雨のち雨
5956     晴一時雨のち曇
6199     雨のち曇のち晴
7006     曇一時雨のち晴
7225    曇一時雷雨のち曇
7585     曇時々雨のち晴
7667     曇時々晴一時雨
7783     雨のち曇一時雨
7919        晴のち雪
7955       雨一時雷雨
8705        曇の

In [290]:
weather_number = compose_processing["weather"].drop_duplicates().nunique()+1#外部データありのとき＋１する nanデータ対応

In [291]:
#天気の辞書の作成
weather_dict={}
for weather_num in range (weather_number):
    weather_dict.update({compose["weather"].drop_duplicates().unique()[weather_num]:weather_num})
#置換
compose_processing["weather"] =compose_processing["weather"].replace(weather_dict)

とりあえず。このまま置いておく  
改良の余地かなりあり

### 6.温度・湿度について

過去のデータの1993年の温度・湿度のデータが存在しない

### attendanceは今回予測するデータ 他に気温・湿度にわずかながらNanデータが存在する
なので,欠損データは今回平均で置き換えることにした　https://newtechnologylifestyle.net/pandasnan/
精査必要あり（10/21）

#### 方法１　平均で置き換える

In [292]:
compose["humidity"]    = compose["humidity"].fillna(compose["humidity"].mean())
compose["temperature"] = compose["temperature"].fillna(compose["temperature"].mean())

#### 方法２ 場所と日にちから考える

In [293]:
compose_processing

,index,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,venue,weather,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,capacity
0,0,59626.0,1,NaN,0,NaN,49,1993-05-15,0,0,NaN,0,0,1993,5,15,19,29,5,48000
1,1,14126.0,5,NaN,1,NaN,50,1993-05-16,1,0,NaN,1,0,1993,5,16,13,5,6,15454
2,2,11875.0,7,NaN,2,NaN,51,1993-05-16,1,0,NaN,2,0,1993,5,16,13,59,6,33000
3,3,10898.0,8,NaN,3,NaN,52,1993-05-16,1,0,NaN,3,0,1993,5,16,16,0,6,40728
4,4,19580.0,6,NaN,4,NaN,53,1993-05-16,1,0,NaN,4,0,1993,5,16,19,4,6,21000
5,5,9183.0,2,NaN,5,NaN,58,1993-05-19,0,1,NaN,5,1,1993,5,19,18,58,2,20339
6,6,9259.0,8,NaN,6,NaN,56,1993-05-19,0,1,NaN,6,0,1993,5,19,19,0,2,21500
7,7,10332.0,4,NaN,1,NaN,57,1993-05-19,0,1,NaN,1,0,1993,5,19,19,0,2,15454
8,8,9403.0,1,NaN,3,NaN,54,1993-05-19,0,1,NaN,3,0,1993,5,19,19,1,2,40728
9,9,46959.0,0,NaN,7,NaN,55,1993-05-19,0,1,NaN,0,0,1993,5,19,19,2,2,48000


### 2.放送会社　ラベリング  

In [294]:
#nanを0に変換
compose_processing["broadcasters"] =compose_processing["broadcasters"].fillna("放送なし")

In [295]:
compose_processing["broadcasters"] 

0                放送なし
1                放送なし
2                放送なし
3                放送なし
4                放送なし
5                放送なし
6                放送なし
7                放送なし
8                放送なし
9                放送なし
10               放送なし
11               放送なし
12               放送なし
13               放送なし
14               放送なし
15               放送なし
16               放送なし
17               放送なし
18               放送なし
19               放送なし
20               放送なし
21               放送なし
22               放送なし
23               放送なし
24               放送なし
25               放送なし
26               放送なし
27               放送なし
28               放送なし
29               放送なし
            ...      
8980             DAZN
8981             DAZN
8982             DAZN
8983             DAZN
8984       DAZN/サンテレビ
8985             DAZN
8986     DAZN/NHK BS1
8987    DAZN/TOKYO MX
8988             DAZN
8989             DAZN
8990             DAZN
8991             DAZN
8992       DAZN/NHK仙台
8993    DAZN/TOKYO MX
8994      

In [296]:
#重複がない数をカウントする
dirty_casters =set()
for casters in compose_processing["broadcasters"].str.split("/"):
    dirty_casters.update(casters)

In [222]:
# ※以降はチャンネルと関係ない
# スカパー!関連が!がなかったりSportsのスペルや大文字小文字がぐちゃぐちゃになっていて酷い
dirty_casters

{'BS',
 'BS(録)',
 'BS-TBS',
 'BS-TBS(録)',
 'BS-i',
 'BS-i(録)',
 'BS※',
 'BSーTBS',
 'BSーTBS(録)',
 'BSーi',
 'DAZN',
 'DAZN ※AFCチャンピオンズリーグ2017の影響により、G大阪vs鹿島は7月5日(水)に開催',
 'DAZN ※AFCチャンピオンズリーグ2017の影響により、川崎Fvs浦和は7月5日(水)に開催',
 'J SPORTS',
 'J SPORTS ESPN',
 'J SPORTS(録)',
 'J SPORTS2',
 'KBS京都',
 'KBS京都(録)',
 'MXテレビ',
 'MXテレビ(録)',
 'MXテレビ*',
 'NHK BS1',
 'NHK BS1 ※4 AFCチャンピオンズリーグ2016の影響により、浦和vsFC東京は6月22日(水)に開催',
 'NHK BS1 ※6 浦和がスルガ銀行チャンピオンシップ2017SAITAMAに出場のため、7月22日(土)に開催',
 'NHK BS1 ※AFCチャンピオンズリーグ2018の影響により、C大阪vs鹿島は7月25日(水)に開催',
 'NHK BS1(101ch)',
 'NHK BS1(102ch)',
 'NHK BS1(録)',
 'NHK-BSハイビジョン',
 'NHK京都',
 'NHK仙台',
 'NHK仙台 ※ AFCチャンピオンズリーグ2015準決勝の影響により、10月25日(日)に開催。',
 'NHK仙台(東北ブロック)',
 'NHK佐賀',
 'NHK名古屋',
 'NHK名古屋(録)',
 'NHK大分',
 'NHK大阪',
 'NHK大阪(近畿ブロック)',
 'NHK大阪(録)',
 'NHK山形',
 'NHK岐阜',
 'NHK岩手',
 'NHK広島',
 'NHK徳島',
 'NHK新潟',
 'NHK新潟*',
 'NHK札幌',
 'NHK水戸',
 'NHK水戸*',
 'NHK津',
 'NHK甲府',
 'NHK神戸',
 'NHK神戸(近畿ブロック)',
 'NHK神戸(録)',
 'NHK福岡',
 'NHK福島',
 'NHK秋田',
 'NHK総合',
 'NHK総合(録)',
 'NHK長崎',

In [223]:
len(dirty_casters)

336

方針:  
 ()と※より後ろを消す  
 小文字を大文字に統一  
 ーを-に変換する  
 スカパー派生は全てスカパーに統一  

In [224]:
def cleanify(caster):
    caster = caster.upper()
    caster = re.sub("\(.+\)|\*", "", caster)
    caster = caster.split("※")[0]\
               .rstrip(" ")\
               .replace("ー", "-")\
               .replace(" ", "_")
    if caster[:4] == "スカパ-":
        caster = "スカパー"
    return caster

<input>:3: DeprecationWarning:

invalid escape sequence \(

<input>:3: DeprecationWarning:

invalid escape sequence \(

<input>:3: DeprecationWarning:

invalid escape sequence \(

<input>:3: DeprecationWarning:

invalid escape sequence \(

<input>:3: DeprecationWarning:

invalid escape sequence \(

<input>:3: DeprecationWarning:

invalid escape sequence \(

<input>:3: DeprecationWarning:

invalid escape sequence \(

<input>:3: DeprecationWarning:

invalid escape sequence \(

<input>:3: DeprecationWarning:

invalid escape sequence \(

<input>:3: DeprecationWarning:

invalid escape sequence \(

<input>:3: DeprecationWarning:

invalid escape sequence \(

<input>:3: DeprecationWarning:

invalid escape sequence \(

<ipython-input-224-34d4891d9261>:3: DeprecationWarning:

invalid escape sequence \(



In [ ]:
for i in range(len(compose_processing["broadcasters"] )):
    compose_processing["broadcasters"][i] = cleanify(compose_processing["broadcasters"][i])

C:\Users\Mustang8\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [ ]:
compose_processing["broadcasters"]

In [198]:
tmp =compose_processing["broadcasters"].str.split("/")\
                              .apply(lambda casters: [cleanify(caster) for caster in casters])
compose_processing["broadcasters"] = tmp

TypeError: 'float' object is not iterable

In [199]:
for caster in clean_casters:
    compose_processing["broadcasters_{}".format(caster)] =compose_processing["broadcasters"].apply(lambda casters: caster in casters)

In [194]:
from collections import Counter

In [300]:
#放送会社の辞書の作成
broadcasters_number = compose_processing["broadcasters"].drop_duplicates().nunique()#外部データありのとき＋１する nanデータ対応
broadcasters_dict={}
for broadcasters_num in range (broadcasters_number):
    broadcasters_dict.update({compose_processing["broadcasters"].drop_duplicates().unique()[broadcasters_num]:broadcasters_num})
#置換
compose_processing["broadcasters"] =compose_processing["broadcasters"].replace(broadcasters_dict)

In [144]:
#nanデータ　平均で置換
compose_processing["weather"]   = compose_processing["weather"].fillna(compose_processing["weather"].mean())

### 点数と勝敗だけいれる　外部データと統合

In [304]:
compose_processing1 = compose_processing.drop("index",axis=1)

In [305]:
compose_processing2 = pd.merge(compose_processing1 , compose_match_reports2 , on="id" ,how="left")

In [306]:
compose_match_reports2.shape

(14376, 4)

In [307]:
compose_match_reports2.head(500)

,away_team_score,home_team_score,home_team_win,id
0,2,1,False,49
1,2,3,True,50
2,1,2,True,51
3,0,5,True,52
4,0,1,True,53
5,0,1,True,58
6,3,0,False,56
7,0,1,True,57
8,2,3,True,54
9,1,2,True,55


In [308]:
compose_processing2.head(500)

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,...,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,capacity,away_team_score,home_team_score,home_team_win
0,59626.0,1,0,0,NaN,49,1993-05-15,0,0,NaN,...,1993,5,15,19,29,5,48000,2.0,1.0,False
1,14126.0,5,0,1,NaN,50,1993-05-16,1,0,NaN,...,1993,5,16,13,5,6,15454,2.0,3.0,True
2,11875.0,7,0,2,NaN,51,1993-05-16,1,0,NaN,...,1993,5,16,13,59,6,33000,1.0,2.0,True
3,10898.0,8,0,3,NaN,52,1993-05-16,1,0,NaN,...,1993,5,16,16,0,6,40728,0.0,5.0,True
4,19580.0,6,0,4,NaN,53,1993-05-16,1,0,NaN,...,1993,5,16,19,4,6,21000,0.0,1.0,True
5,9183.0,2,0,5,NaN,58,1993-05-19,0,1,NaN,...,1993,5,19,18,58,2,20339,0.0,1.0,True
6,9259.0,8,0,6,NaN,56,1993-05-19,0,1,NaN,...,1993,5,19,19,0,2,21500,3.0,0.0,False
7,10332.0,4,0,1,NaN,57,1993-05-19,0,1,NaN,...,1993,5,19,19,0,2,15454,0.0,1.0,True
8,9403.0,1,0,3,NaN,54,1993-05-19,0,1,NaN,...,1993,5,19,19,1,2,40728,2.0,3.0,True
9,46959.0,0,0,7,NaN,55,1993-05-19,0,1,NaN,...,1993,5,19,19,2,2,48000,1.0,2.0,True


In [309]:
print(compose_processing1.shape)
print(compose_processing2.shape)

(9010, 19)
(9010, 22)


In [310]:
compose_processing2

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,...,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,capacity,away_team_score,home_team_score,home_team_win
0,59626.0,1,0,0,NaN,49,1993-05-15,0,0,NaN,...,1993,5,15,19,29,5,48000,2.0,1.0,False
1,14126.0,5,0,1,NaN,50,1993-05-16,1,0,NaN,...,1993,5,16,13,5,6,15454,2.0,3.0,True
2,11875.0,7,0,2,NaN,51,1993-05-16,1,0,NaN,...,1993,5,16,13,59,6,33000,1.0,2.0,True
3,10898.0,8,0,3,NaN,52,1993-05-16,1,0,NaN,...,1993,5,16,16,0,6,40728,0.0,5.0,True
4,19580.0,6,0,4,NaN,53,1993-05-16,1,0,NaN,...,1993,5,16,19,4,6,21000,0.0,1.0,True
5,9183.0,2,0,5,NaN,58,1993-05-19,0,1,NaN,...,1993,5,19,18,58,2,20339,0.0,1.0,True
6,9259.0,8,0,6,NaN,56,1993-05-19,0,1,NaN,...,1993,5,19,19,0,2,21500,3.0,0.0,False
7,10332.0,4,0,1,NaN,57,1993-05-19,0,1,NaN,...,1993,5,19,19,0,2,15454,0.0,1.0,True
8,9403.0,1,0,3,NaN,54,1993-05-19,0,1,NaN,...,1993,5,19,19,1,2,40728,2.0,3.0,True
9,46959.0,0,0,7,NaN,55,1993-05-19,0,1,NaN,...,1993,5,19,19,2,2,48000,1.0,2.0,True


おかしい　数が合わない…

In [311]:
print(compose_processing1["id"].value_counts())
print(len(compose_processing1["id"].value_counts()))

6129     1
15712    1
527      1
13463    1
5174     1
8656     1
11824    1
1270     1
7416     1
51       1
6443     1
11669    1
5890     1
15713    1
1464     1
11668    1
8949     1
8519     1
11683    1
4601     1
8933     1
3841     1
5470     1
11678    1
1387     1
8544     1
8657     1
15715    1
15768    1
5846     1
        ..
11625    1
11624    1
5466     1
11622    1
11620    1
11645    1
11619    1
572      1
11617    1
11616    1
2232     1
8769     1
11628    1
11629    1
11630    1
11631    1
11632    1
5370     1
11634    1
8787     1
3104     1
11637    1
11638    1
11639    1
11640    1
11641    1
11642    1
11643    1
1233     1
7213     1
Name: id, Length: 9010, dtype: int64
9010


In [312]:
print(compose_match_reports["id"].value_counts())
print(len(compose_match_reports["id"].value_counts()))

11070    2
14268    2
11686    2
15780    2
14284    2
10190    2
11670    2
15764    2
10206    2
11654    2
15748    2
10222    2
11638    2
15732    2
10238    2
11622    2
15716    2
10254    2
15700    2
10174    2
15796    2
9263     2
11702    2
11766    2
14204    2
10110    2
15860    2
17899    2
11750    2
14220    2
        ..
3784     1
16062    1
11964    1
5819     1
7866     1
3768     1
3832     1
7930     1
5883     1
12028    1
3896     1
18225    1
16174    1
12076    1
5931     1
7978     1
3880     1
18209    1
16158    1
12060    1
5915     1
7962     1
3864     1
16142    1
12044    1
5899     1
7946     1
3848     1
16126    1
2049     1
Name: id, Length: 14376, dtype: int64
14376


まさかのmatch_reportsに重複アリ

In [313]:
#重複行削除
compose_match_reports.drop_duplicates(inplace = True,subset="id")

In [314]:
compose_match_reports.shape

(14376, 4)

In [315]:
compose_processing3 = compose_processing1.merge(compose_match_reports2[["id", "home_team_win", "home_team_score", "away_team_score"]] , how="left",
                          on="id")

In [316]:
compose_processing3.shape 

(9010, 22)

In [317]:
compose_processing3.head(500)

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,...,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,capacity,home_team_win,home_team_score,away_team_score
0,59626.0,1,0,0,NaN,49,1993-05-15,0,0,NaN,...,1993,5,15,19,29,5,48000,False,1.0,2.0
1,14126.0,5,0,1,NaN,50,1993-05-16,1,0,NaN,...,1993,5,16,13,5,6,15454,True,3.0,2.0
2,11875.0,7,0,2,NaN,51,1993-05-16,1,0,NaN,...,1993,5,16,13,59,6,33000,True,2.0,1.0
3,10898.0,8,0,3,NaN,52,1993-05-16,1,0,NaN,...,1993,5,16,16,0,6,40728,True,5.0,0.0
4,19580.0,6,0,4,NaN,53,1993-05-16,1,0,NaN,...,1993,5,16,19,4,6,21000,True,1.0,0.0
5,9183.0,2,0,5,NaN,58,1993-05-19,0,1,NaN,...,1993,5,19,18,58,2,20339,True,1.0,0.0
6,9259.0,8,0,6,NaN,56,1993-05-19,0,1,NaN,...,1993,5,19,19,0,2,21500,False,0.0,3.0
7,10332.0,4,0,1,NaN,57,1993-05-19,0,1,NaN,...,1993,5,19,19,0,2,15454,True,1.0,0.0
8,9403.0,1,0,3,NaN,54,1993-05-19,0,1,NaN,...,1993,5,19,19,1,2,40728,True,3.0,2.0
9,46959.0,0,0,7,NaN,55,1993-05-19,0,1,NaN,...,1993,5,19,19,2,2,48000,True,2.0,1.0


In [318]:
#重複行削除
compose_processing4= compose_processing3.drop_duplicates(inplace = False,subset="id")

In [319]:
compose_processing4.shape 

(9010, 22)

数がようやくあった

In [320]:
compose_processing3

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,...,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,capacity,home_team_win,home_team_score,away_team_score
0,59626.0,1,0,0,NaN,49,1993-05-15,0,0,NaN,...,1993,5,15,19,29,5,48000,False,1.0,2.0
1,14126.0,5,0,1,NaN,50,1993-05-16,1,0,NaN,...,1993,5,16,13,5,6,15454,True,3.0,2.0
2,11875.0,7,0,2,NaN,51,1993-05-16,1,0,NaN,...,1993,5,16,13,59,6,33000,True,2.0,1.0
3,10898.0,8,0,3,NaN,52,1993-05-16,1,0,NaN,...,1993,5,16,16,0,6,40728,True,5.0,0.0
4,19580.0,6,0,4,NaN,53,1993-05-16,1,0,NaN,...,1993,5,16,19,4,6,21000,True,1.0,0.0
5,9183.0,2,0,5,NaN,58,1993-05-19,0,1,NaN,...,1993,5,19,18,58,2,20339,True,1.0,0.0
6,9259.0,8,0,6,NaN,56,1993-05-19,0,1,NaN,...,1993,5,19,19,0,2,21500,False,0.0,3.0
7,10332.0,4,0,1,NaN,57,1993-05-19,0,1,NaN,...,1993,5,19,19,0,2,15454,True,1.0,0.0
8,9403.0,1,0,3,NaN,54,1993-05-19,0,1,NaN,...,1993,5,19,19,1,2,40728,True,3.0,2.0
9,46959.0,0,0,7,NaN,55,1993-05-19,0,1,NaN,...,1993,5,19,19,2,2,48000,True,2.0,1.0


In [179]:
compose_processing2 = compose_processing2.rename(columns={"home_team_score_y":"home_team_score"})
compose_processing2 = compose_processing2.rename(columns={"away_team_score_y":"away_team_score"})
compose_processing2 = compose_processing2.rename(columns={"home_team_win_y":"home_team_win"})

### データをソート　match-dateでソートした

In [322]:
compose_processing4.sort_values(by = "match_date")
compose_processing4.reset_index

<bound method DataFrame.reset_index of       attendance  away_team  broadcasters  home_team  humidity     id  \
0        59626.0          1             0          0       NaN     49   
1        14126.0          5             0          1       NaN     50   
2        11875.0          7             0          2       NaN     51   
3        10898.0          8             0          3       NaN     52   
4        19580.0          6             0          4       NaN     53   
5         9183.0          2             0          5       NaN     58   
6         9259.0          8             0          6       NaN     56   
7        10332.0          4             0          1       NaN     57   
8         9403.0          1             0          3       NaN     54   
9        46959.0          0             0          7       NaN     55   
10       11267.0          1             0          8       NaN     59   
11       36863.0          0             0          2       NaN     61   
12        74

In [323]:
compose_processing4

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,...,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,capacity,home_team_win,home_team_score,away_team_score
0,59626.0,1,0,0,NaN,49,1993-05-15,0,0,NaN,...,1993,5,15,19,29,5,48000,False,1.0,2.0
1,14126.0,5,0,1,NaN,50,1993-05-16,1,0,NaN,...,1993,5,16,13,5,6,15454,True,3.0,2.0
2,11875.0,7,0,2,NaN,51,1993-05-16,1,0,NaN,...,1993,5,16,13,59,6,33000,True,2.0,1.0
3,10898.0,8,0,3,NaN,52,1993-05-16,1,0,NaN,...,1993,5,16,16,0,6,40728,True,5.0,0.0
4,19580.0,6,0,4,NaN,53,1993-05-16,1,0,NaN,...,1993,5,16,19,4,6,21000,True,1.0,0.0
5,9183.0,2,0,5,NaN,58,1993-05-19,0,1,NaN,...,1993,5,19,18,58,2,20339,True,1.0,0.0
6,9259.0,8,0,6,NaN,56,1993-05-19,0,1,NaN,...,1993,5,19,19,0,2,21500,False,0.0,3.0
7,10332.0,4,0,1,NaN,57,1993-05-19,0,1,NaN,...,1993,5,19,19,0,2,15454,True,1.0,0.0
8,9403.0,1,0,3,NaN,54,1993-05-19,0,1,NaN,...,1993,5,19,19,1,2,40728,True,3.0,2.0
9,46959.0,0,0,7,NaN,55,1993-05-19,0,1,NaN,...,1993,5,19,19,2,2,48000,True,2.0,1.0


In [324]:
print(compose_processing3.match_date)

0      1993-05-15
1      1993-05-16
2      1993-05-16
3      1993-05-16
4      1993-05-16
5      1993-05-19
6      1993-05-19
7      1993-05-19
8      1993-05-19
9      1993-05-19
10     1993-05-22
11     1993-05-22
12     1993-05-22
13     1993-05-22
14     1993-05-22
15     1993-05-26
16     1993-05-26
17     1993-05-26
18     1993-05-26
19     1993-05-26
20     1993-05-29
21     1993-05-29
22     1993-05-29
23     1993-05-29
24     1993-05-29
25     1993-06-02
26     1993-06-02
27     1993-06-02
28     1993-06-02
29     1993-06-02
          ...    
8980   2018-07-18
8981   2018-07-18
8982   2018-07-18
8983   2018-07-22
8984   2018-07-22
8985   2018-07-22
8986   2018-07-22
8987   2018-07-22
8988   2018-07-22
8989   2018-07-22
8990   2018-07-22
8991   2018-07-22
8992   2018-11-24
8993   2018-11-24
8994   2018-11-24
8995   2018-11-24
8996   2018-11-24
8997   2018-11-24
8998   2018-11-24
8999   2018-11-24
9000   2018-11-24
9001   2018-12-01
9002   2018-12-01
9003   2018-12-01
9004   201

In [325]:
compose_processing.to_csv("compose_processing.csv",header=True,index=None)

In [326]:
compose_processing2.to_csv("compose_processing2.csv",header=True,index=None)
compose_processing3.to_csv("compose_processing3.csv",header=True,index=None)
compose_processing4.to_csv("compose_processing4.csv",header=True,index=None)

# 工事中　one-hot-vector

In [327]:
pd.set_option('display.max_columns', 100)

# 天気をone-hot-vectorする

参考URL：  
https://www.haya-programming.com/entry/2018/06/09/170504

In [328]:
from sklearn.preprocessing import OneHotEncoder

In [329]:
weather_vector=OneHotEncoder().fit_transform(compose_processing4["weather"].reshape(-1, 1)).A

C:\Users\Mustang8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



In [330]:
weather_vector.shape

(9010, 73)

In [331]:
weather_vector = pd.DataFrame(weather_vector)
weather_vector = weather_vector.rename(columns=lambda x: "weather"+ x.astype(str))

In [332]:
compose_processing5 = pd.concat([compose_processing4,weather_vector],axis=1 )

In [333]:
compose_processing5

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,venue,weather,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,capacity,home_team_win,home_team_score,away_team_score,weather0,weather1,weather2,weather3,weather4,weather5,weather6,weather7,weather8,weather9,weather10,weather11,weather12,weather13,weather14,weather15,weather16,weather17,weather18,weather19,weather20,weather21,weather22,weather23,weather24,weather25,weather26,weather27,weather28,weather29,weather30,weather31,weather32,weather33,weather34,weather35,weather36,weather37,weather38,weather39,weather40,weather41,weather42,weather43,weather44,weather45,weather46,weather47,weather48,weather49,weather50,weather51,weather52,weather53,weather54,weather55,weather56,weather57,weather58,weather59,weather60,weather61,weather62,weather63,weather64,weather65,weather66,weather67,weather68,weather69,weather70,weather71,weather72
0,59626.0,1,0,0,NaN,49,1993-05-15,0,0,NaN,0,0,1993,5,15,19,29,5,48000,False,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14126.0,5,0,1,NaN,50,1993-05-16,1,0,NaN,1,0,1993,5,16,13,5,6,15454,True,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11875.0,7,0,2,NaN,51,1993-05-16,1,0,NaN,2,0,1993,5,16,13,59,6,33000,True,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10898.0,8,0,3,NaN,52,1993-05-16,1,0,NaN,3,0,1993,5,16,16,0,6,40728,True,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19580.0,6,0,4,NaN,53,1993-05-16,1,0,NaN,4,0,1993,5,16,19,4,6,21000,True,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,9183.0,2,0,5,NaN,58,1993-05-19,0,1,NaN,5,1,1993,5,19,18,58,2,20339,True,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,9259.0,8,0,6,NaN,56,1993-05-19,0,1,NaN,6,0,1993,5,19,19,0,2,21500,False,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,10332.0,4,0,1,NaN,57,1993-05-19,0,1,NaN,1,0,1993,5,19,19,0,2,15454,True,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9403.0,1,0,3,

In [334]:
compose_processing5.to_csv("compose_processing5.csv",header=True,index=None)

# スタジアムをone-hot-vector

In [335]:
venue_vector=OneHotEncoder().fit_transform(compose_processing4["venue"].reshape(-1, 1)).A

C:\Users\Mustang8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



In [336]:
venue_vector = pd.DataFrame(venue_vector)
venue_vector = venue_vector.rename(columns=lambda x: "venuer"+ x.astype(str))

In [337]:
compose_processing6 = pd.concat([compose_processing5,venue_vector],axis=1 )

In [338]:
compose_processing6.to_csv("compose_processing6.csv",header=True,index=None)

In [356]:
compose_processing6

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,venue,weather,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,capacity,home_team_win,home_team_score,away_team_score,weather0,weather1,weather2,weather3,weather4,weather5,weather6,weather7,weather8,weather9,weather10,weather11,weather12,weather13,weather14,weather15,weather16,weather17,weather18,weather19,weather20,weather21,weather22,weather23,weather24,weather25,weather26,weather27,...,venuer98,venuer99,venuer100,venuer101,venuer102,venuer103,venuer104,venuer105,venuer106,venuer107,venuer108,venuer109,venuer110,venuer111,venuer112,venuer113,venuer114,venuer115,venuer116,venuer117,venuer118,venuer119,venuer120,venuer121,venuer122,venuer123,venuer124,venuer125,venuer126,venuer127,venuer128,venuer129,venuer130,venuer131,venuer132,venuer133,venuer134,venuer135,venuer136,venuer137,venuer138,venuer139,venuer140,venuer141,venuer142,venuer143,venuer144,venuer145,venuer146,venuer147
0,59626.0,1,0,0,NaN,49,1993-05-15,0,0,NaN,0,0,1993,5,15,19,29,5,48000,False,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14126.0,5,0,1,NaN,50,1993-05-16,1,0,NaN,1,0,1993,5,16,13,5,6,15454,True,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11875.0,7,0,2,NaN,51,1993-05-16,1,0,NaN,2,0,1993,5,16,13,59,6,33000,True,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10898.0,8,0,3,NaN,52,1993-05-16,1,0,NaN,3,0,1993,5,16,16,0,6,40728,True,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19580.0,6,0,4,NaN,53,1993-05-16,1,0,NaN,4,0,1993,5,16,19,4,6,21000,True,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,9183.0,2,0,5,NaN,58,1993-05-19,0,1,NaN,5,1,1993,5,19,18,58,2,20339,True,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,9259.0,8,0,6,NaN,56,1993-05-19,0,1,NaN,6,0,1993,5,19,19,0,2,21500,False,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,10332.0,4,0,1,NaN,57,1993-05-19,0,1,NaN,1,0,1993,5,19,19,0,2,15454,True,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

# 放送会社をone-hor-vector

In [357]:
broadcasts_vector=OneHotEncoder().fit_transform(compose_processing4["broadcasters"].reshape(-1, 1)).A

C:\Users\Mustang8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



In [358]:
broadcasts_vector = pd.DataFrame(broadcasts_vector)
broadcasts_vector = broadcasts_vector.rename(columns=lambda x: "broadcasts"+ x.astype(str))

In [359]:
compose_processing7 = pd.concat([compose_processing6,broadcasts_vector],axis=1 )

In [360]:
compose_processing7.to_csv("compose_processing7.csv",header=True,index=None)

# チームをone-hot-vector

In [361]:
team_vector=OneHotEncoder().fit_transform(compose_processing4["home_team"].reshape(-1, 1)).A

C:\Users\Mustang8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



In [362]:
home_team_vector = pd.DataFrame(team_vector)
home_team_vector = home_team_vector.rename(columns=lambda x: "hometeam"+ x.astype(str))

In [363]:
away_team_vector = pd.DataFrame(team_vector)
away_team_vector = away_team_vector.rename(columns=lambda x: "awayteam"+ x.astype(str))

In [364]:
compose_processing8 = pd.concat([compose_processing7,home_team_vector],axis=1 )
compose_processing9 = pd.concat([compose_processing8,away_team_vector],axis=1 )

In [365]:
compose_processing9.to_csv("compose_processing9.csv",header=True,index=None)

# ラウンドをone-hot-vector

In [366]:
round_vector=OneHotEncoder().fit_transform(compose_processing4["round"].reshape(-1, 1)).A

round_vector = pd.DataFrame(round_vector)
round_vector = round_vector.rename(columns=lambda x: "round"+ x.astype(str))

compose_processing10 = pd.concat([compose_processing9,round_vector],axis=1 )

C:\Users\Mustang8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



# セクションをone-hot-vector

In [367]:
section_vector=OneHotEncoder().fit_transform(compose_processing4["section"].reshape(-1, 1)).A

section_vector = pd.DataFrame(section_vector)
section_vector = section_vector.rename(columns=lambda x: "section"+ x.astype(str))

compose_processing11 = pd.concat([compose_processing10,section_vector],axis=1 )

C:\Users\Mustang8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



In [368]:
compose_processing11.head()

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,venue,weather,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,capacity,home_team_win,home_team_score,away_team_score,weather0,weather1,weather2,weather3,weather4,weather5,weather6,weather7,weather8,weather9,weather10,weather11,weather12,weather13,weather14,weather15,weather16,weather17,weather18,weather19,weather20,weather21,weather22,weather23,weather24,weather25,weather26,weather27,...,awayteam31,awayteam32,round0,round1,round2,round3,section0,section1,section2,section3,section4,section5,section6,section7,section8,section9,section10,section11,section12,section13,section14,section15,section16,section17,section18,section19,section20,section21,section22,section23,section24,section25,section26,section27,section28,section29,section30,section31,section32,section33,section34,section35,section36,section37,section38,section39,section40,section41,section42,section43
0,59626.0,1,0,0,NaN,49,1993-05-15,0,0,NaN,0,0,1993,5,15,19,29,5,48000,False,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14126.0,5,0,1,NaN,50,1993-05-16,1,0,NaN,1,0,1993,5,16,13,5,6,15454,True,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11875.0,7,0,2,NaN,51,1993-05-16,1,0,NaN,2,0,1993,5,16,13,59,6,33000,True,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10898.0,8,0,3,NaN,52,1993-05-16,1,0,NaN,3,0,1993,5,16,16,0,6,40728,True,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19580.0,6,0,4,NaN,53,1993-05-16,1,0,NaN,4,0,1993,5,16,19,4,6,21000,True,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [369]:
compose_processing11.to_csv("compose_processing11.csv",header=True,index=None)

In [370]:
compose_processing11

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,venue,weather,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,capacity,home_team_win,home_team_score,away_team_score,weather0,weather1,weather2,weather3,weather4,weather5,weather6,weather7,weather8,weather9,weather10,weather11,weather12,weather13,weather14,weather15,weather16,weather17,weather18,weather19,weather20,weather21,weather22,weather23,weather24,weather25,weather26,weather27,...,awayteam31,awayteam32,round0,round1,round2,round3,section0,section1,section2,section3,section4,section5,section6,section7,section8,section9,section10,section11,section12,section13,section14,section15,section16,section17,section18,section19,section20,section21,section22,section23,section24,section25,section26,section27,section28,section29,section30,section31,section32,section33,section34,section35,section36,section37,section38,section39,section40,section41,section42,section43
0,59626.0,1,0,0,NaN,49,1993-05-15,0,0,NaN,0,0,1993,5,15,19,29,5,48000,False,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14126.0,5,0,1,NaN,50,1993-05-16,1,0,NaN,1,0,1993,5,16,13,5,6,15454,True,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11875.0,7,0,2,NaN,51,1993-05-16,1,0,NaN,2,0,1993,5,16,13,59,6,33000,True,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10898.0,8,0,3,NaN,52,1993-05-16,1,0,NaN,3,0,1993,5,16,16,0,6,40728,True,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19580.0,6,0,4,NaN,53,1993-05-16,1,0,NaN,4,0,1993,5,16,19,4,6,21000,True,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,9183.0,2,0,5,NaN,58,1993-05-19,0,1,NaN,5,1,1993,5,19,18,58,2,20339,True,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,9259.0,8,0,6,NaN,56,1993-05-19,0,1,NaN,6,0,1993,5,19,19,0,2,21500,False,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,10332.0,4,0,1,NaN,57,1993-05-19,0,1,NaN,1,0,1993,5,19,19,0,2,15454,True,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

参考  
https://qiita.com/uratatsu/items/8bedbf91e22f90b6e64b  
http://sinhrks.hatenablog.com/entry/2015/01/28/073327  
https://note.nkmk.me/python-pandas-merge-join/  
